# Capítulo 2. Proyecto de Machine Learning End-to-End

# Indice

+ [Trabajando con datos reales](#Trabajando-con-datos-reales)
+ [Visión general](#Visión-general)
 + [Enmarcar el problema](#Enmarcar-el-problema)
   + [Aprendizaje supervisado](#Aprendizaje-supervisado)

En este capítulo, iremos a través de un ejemplo de proyecto end-to-end, pretendiendo ser un científico de datos recién contratado por una empresa inmobiliaria. Estos son los principales pasos que daremos:

- Echar un vistazo global
- Recopilar los datos
- Descubrir y visualizar los datos para obtener ideas (*insights*)
- Preparar los datos para los algoritmos de Machine Learning
- Seleccionar un modelo y entrenarlo
- Afinar el modelo
- Presentar la solución
- Implantar, monitorizar y mantener el sistema

## Trabajando con datos reales

Cuando estamos aprendiendo sobre Machine Learning lo mejor es experimentar con datos reales, no solo conjuntos de datos artificiales. Afortunadamente, existen miles de conjuntos de datos abiertos para elegir que abarcan todo tipo de dominios. He aquí algunos sitios donde podemos obtener datos:

- Repositorios populares de datos abiertos:
 - [Repositorio de Machine Learning UC Irvine](http://archive.ics.uci.edu/ml/)
 - [Conjunto de datos de Kaggle](https://www.kaggle.com/datasets)
 - [Conjunto de datos AWS de Amazon](https://registry.opendata.aws/)
 
- Metaportales (lista abierta de repositorios):
 - <http://dataportals.org>
 - <http://opendatamonitor.eu>
 - <http://quandl.com>
 
- Otras páginas que listan muchos repositorios populares de datos abiertos:
 - [Conjuntos de datos de Machine Learning de Wikipedia](https://homl.info/9)
 - [Quora.com](https://homl.info/10)
 - [Conjuntos de datos de subreddit](https://www.reddit.com/r/datasets)

En este capítulo utilizaremos el conjunto de datos de Precios de Viviendas de California del repositorio StatLib (ver siguiente figura). Este repositorio se basó en datos del censo de California de 1990. No es que sea exactamente reciente, pero tiene muchas cualidades para el aprendizaje. También agregaremos atributos categóricos y eliminaremos algunas características con propósitos educativos.

![california_dataset](images/ch02/california_dataset.png)

## Visión general

¡Bienvenido a Machine Learning Housing Corporation! La primera tarea que debes realziar es crear un modelo de precios de viviendas en California usando los datos censales de California. Estos datos tienen métricas tales como la población, ingresos medios, precio medio de la vivienda, y así por cada grupo de bloques en California. Los grupos de bloques son pequeñas unidades geográficas para las cuales la Oficina del Censo de EEUU publica datos de muestra (un grupo de bloque tiene típicamente una población de 600 a 3.000 personas). Para abreviar los llamaremos "distritos".

Tu modelo debe aprender de estos datos y ser capaz de predecir el precio medio de la vivienda de cualquier distrito, dadas todas las otras métricas.

Como eres un científico de datos bien organizado, la primera cosa que haces es sacar tu checklist de de proyectos de Machine Learning. Debería funcionar razonablemente bien con la mayoría de proyectos de Machine Learning pero asegúrate de adaptarlo a tus necesidades. En este capítulo revisaremos muchos de los elementos de la checklist, pero nos saltaremos otros, bien porque son autoexplicativos o porque serán discutidos en capítulos posteriores.

### Enmarcar el problema

La primera pregunta que debemos hacer a nuestro jefe es cuál es exactamente el objetivo de negocio; contruir un modelo probablemente no sea el objetivo final. ¿Cómo espera la empresa usar y beneficiarse de este modelo? Es importante porque determinará como enmarcaremos el problema, qué algoritmos seleccionaremos, qué medidas de la ejecución usaremos para evaluar nuestro modelo y cuánto esfuerzo deberemos dedicar a su ajuste.

Nuestro jefe responde que las salidas del modelo (la predicción del precio medio de la vivienda de un distrito) alimentará otro sistema de Machine Learning (como se ve en la siguiente figura), junto con otras muchas *señales* [*una pieza de información alimenta a un sistema de Machine Learning se denomina a menudo *señal*, en referencia a la teoría de la información de Shannon: queremos una alta relación señal/ruido*].El sistema siguiente en la cadena determinará si conviene invertir o no en una determinada área. Hacer esto bien es crítico, ya que afecta directamente a los ingresos.

![pipeline_real_state](images/ch02/pipeline_real_state.png)

---
**Pipelines**

Una secuencia de *componentes* procesando datos se denomina un *pipeline* (tubería) de datos. Los pipelines son muy comunes en los sistema de Machine Learning, dado que existen muchos datos que manipular y muchas transformaciones que aplicar.

Normalmente los componentes se ejecutan asíncronamente. Cada componente extrae una gran cantidad de datos, los procesa y escupe el resultado a otro almacén de datos. Entonces, algún tiempo después el siguiente componente en el pipeline extrae estos datos y los escupe en su propia salida, y así sucesivamente. Cada componente bastante autocontenido: el interfaz entre componenetes es simplemente el almacén de datos. Esto hace el sistema bastante fácil de entender (con la ayuda de un gráfico de flujo de datos) y diferentes equipos se pueden enfocar en diferentes componentes. Además, si un componente se cae, el siguiente componente en la cadena a menudo puede continuar ejecutándose normalmente (al menos por un tiempo) tan solo usando la última salida del componente caido. Esto hace que la arquitectura sea bastante robusta.

Por otro lado, un componente caido puede pasar desapercibido por algún tiempo si no está implementada la apropiada monitorización. Los datos se vuelven obsoletos y la ejecución global del sistema se deteriora.

---